In [2]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.0 MB/s eta 0:00:00


In [33]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn import metrics
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import warnings
from sklearn.metrics import r2_score

warnings.filterwarnings('ignore')
import catboost as cb
from catboost import CatBoostRegressor

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [34]:
data = pd.read_csv('/content/drive/MyDrive/!mlData/info558Kaggle/df-train_1000x12.csv')
test_data = pd.read_csv('/content/drive/MyDrive/!mlData/info558Kaggle/df-test_200x11.csv')
print(data.dtypes)
print(data.isnull().sum())


homeid                int64
feat_yrbuilt          int64
feat_nrooms           int64
feat_nbed             int64
feat_nbath          float64
feat_sqft             int64
feat_proptype        object
feat_basement        object
feat_stories         object
feat_solar           object
feat_nsales           int64
target_homevalue      int64
dtype: object
homeid              0
feat_yrbuilt        0
feat_nrooms         0
feat_nbed           0
feat_nbath          0
feat_sqft           0
feat_proptype       0
feat_basement       0
feat_stories        0
feat_solar          0
feat_nsales         0
target_homevalue    0
dtype: int64


In [53]:
def encode_cols(input_data):
  encoder = LabelEncoder()
  for col in input_data.columns:
      if input_data[col].dtype == 'object':
          input_data[col] = encoder.fit_transform(input_data[col])
  print(input_data.head())
  return input_data

In [54]:
data = encode_cols(data)

   homeid  feat_yrbuilt  feat_nrooms  feat_nbed  feat_nbath  feat_sqft  \
0    5936          2024           16          2         7.0       3456   
1    7314          1899           13          2         1.0        500   
2    5448          2024            7          2         9.0       7414   
3    8560          2007           11          2         5.5       6619   
4    5783          2024           15          3         4.5       3226   

   feat_proptype  feat_basement  feat_stories  feat_solar  feat_nsales  \
0              0              0             1           0            2   
1              0              0             1           0            1   
2              2              0             1           0            2   
3              2              0             1           0            3   
4              1              0             0           0            1   

   target_homevalue  
0            151512  
1            356344  
2             78112  
3            872768  


In [36]:
X=data.drop(['target_homevalue','homeid'],axis=1)
y=data['target_homevalue']
print(X.head())
print(y.head())
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=412)

train_dataset = cb.Pool(X_train, y_train)
test_dataset = cb.Pool(X_test, y_test)

   feat_yrbuilt  feat_nrooms  feat_nbed  feat_nbath  feat_sqft  feat_proptype  \
0          2024           16          2         7.0       3456              0   
1          1899           13          2         1.0        500              0   
2          2024            7          2         9.0       7414              2   
3          2007           11          2         5.5       6619              2   
4          2024           15          3         4.5       3226              1   

   feat_basement  feat_stories  feat_solar  feat_nsales  
0              0             1           0            2  
1              0             1           0            1  
2              0             1           0            2  
3              0             1           0            3  
4              0             0           0            1  
0    151512
1    356344
2     78112
3    872768
4    946086
Name: target_homevalue, dtype: int64


In [37]:
catBoostModel = cb.CatBoostRegressor(loss_function="RMSE", task_type="GPU")

In [38]:
grid = {'iterations': [25, 50, 100, 150],
        'learning_rate': [0.001, 0.03, 0.1],
        'depth': [2, 4, 6, 8, 10],
        'l2_leaf_reg': [0.2, 0.5, 1, 3, 10, 15]}
catBoostModel.grid_search(grid, train_dataset)

Streaming output truncated to the last 5000 lines.
bestTest = 398286.6067
bestIteration = 162
66:	loss: 398286.6067063	best: 395590.3316109 (31)	total: 1m 38s	remaining: 42.6s
0:	learn: 591361.5515131	test: 584005.0968326	best: 584005.0968326 (0)	total: 9.29ms	remaining: 1.85s
1:	learn: 558188.4489124	test: 554504.1787024	best: 554504.1787024 (1)	total: 17.8ms	remaining: 1.76s
2:	learn: 530661.2137958	test: 531640.6677236	best: 531640.6677236 (2)	total: 37.8ms	remaining: 2.48s
3:	learn: 507779.9848596	test: 514045.1697133	best: 514045.1697133 (3)	total: 43.6ms	remaining: 2.13s
4:	learn: 484446.2453763	test: 497173.0912107	best: 497173.0912107 (4)	total: 54.3ms	remaining: 2.12s
5:	learn: 463050.5185260	test: 483780.8048445	best: 483780.8048445 (5)	total: 62.6ms	remaining: 2.02s
6:	learn: 447942.5352859	test: 471971.1978195	best: 471971.1978195 (6)	total: 71.2ms	remaining: 1.96s
7:	learn: 432518.6783740	test: 460417.8468687	best: 460417.8468687 (7)	total: 79.4ms	remaining: 1.9s
8:	learn:

{'params': {'depth': 4,
  'learning_rate': 0.1,
  'l2_leaf_reg': 3,
  'iterations': 100},
 'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               45,

In [41]:
pred = catBoostModel.predict(X_test)
rmse = (np.sqrt(mean_squared_error(y_test, pred)))
r2 = r2_score(y_test, pred)
print("Testing performance")
print("RMSE: {:.2f}".format(rmse))
print("R2: {:.2f}".format(r2))

Testing performance
RMSE: 433315.18
R2: 0.07


In [55]:
def submit_csv(test_data, model):
  test_data_input = test_data.drop(['homeid'], axis=1)
  test_data_input = encode_cols(test_data_input)
  test_pred = model.predict(test_data_input)
  submission = pd.DataFrame({'homeid': test_data['homeid'], 'target_homevalue': test_pred})
  submission.to_csv('submission.csv', index=False)

In [57]:
submit_csv(test_data, model)

   feat_yrbuilt  feat_nrooms  feat_nbed  feat_nbath  feat_sqft  feat_proptype  \
0          1734            8          7         6.5       4500              1   
1          2024           10          3         4.0       4470              0   
2          1692           13          2         3.0       3985              2   
3          2024            8          2         6.0       2079              2   
4          2024            3          2         7.5        500              1   

   feat_basement  feat_stories  feat_solar  feat_nsales  
0              1             0           0            2  
1              0             1           0            0  
2              0             1           0            2  
3              0             1           0            3  
4              1             0           0            2  
200 200


Linear Regression performance

In [28]:
def linear_reg_results(X_train, y_train, X_test, y_test):
  from sklearn.linear_model import LinearRegression
  from sklearn.metrics import mean_squared_error, r2_score
  reg = LinearRegression().fit(X_train, y_train)
  score = reg.score(X_train, y_train)
  y_pred = reg.predict(X_test)
  rmse = (np.sqrt(mean_squared_error(y_test, y_pred)))
  r2 = r2_score(y_test, y_pred)
  print("Testing performance")
  print("RMSE: {:.2f}".format(rmse))
  print("R2: {:.2f}".format(r2))

In [29]:
linear_reg_results(X_train, y_train, X_test, y_test)

Testing performance
RMSE: 420906.43
R2: 0.13
